<h1>Parameter Setting</h1>

Generation of test cosmologies with varied parameters requires criteria to be set for determining what parameter combinations are available. This notebook will specifically be looking for combinations of $w_0$, $w_a$, $h$, $\Omega_b$, and $\Omega_m$ (or equivalently $\Omega_{cdm}$ for a universe with only baryonic and cold dark matter contributions to $\Omega_m$) for a flat universe with a nonzero $w_a$ component in the equation-of-state parameter for dark energy.

To avoid unphysical perturbations from a fluid model where $w(z) = -1$ at any redshift, the constant component of $w$ is taken to be
$$w_0 = -1 + \epsilon,$$
where the sign of $\epsilon$ is the sign of $w_a$. The magnitude of $\epsilon$ is taken to keep the effects of a nonzero $\epsilon$ small compared to the effects of $w_a$. The magnitude of $w_a$ is taken to induce a deviation from a flat cosmological constant model on the same order as a model with $w(z) = -1 \pm 0.1$.

Determination of values for potential $\Omega_m$, and $\Omega_b$ values are made by maintaining constant values of $h^2 \Omega_m$ and $h^2 \Omega_b$ for potential $h$ values \[1]. A single set of $h$, $\Omega_b$, and $\Omega_m$ is chosen by holding the distance of the last scattering constant. The redshift at last scattering is aprroximated as $z_* = 1091$, and baseline values for $h^2 \Omega_m$, $h^2 \Omega_b$, and $D_*$ are taken from a flat model with a cosmological constant.

\[[1](http://arxiv.org/abs/1201.2434v2)] Weinberg, D., Mortonson, M., Eisenstein, D., Hirata, C., Riess, A., and Rozo, E. (2013) "Observational Probes of Cosmic Acceleration."

In [1]:
%pylab inline
import scipy.integrate
from astropy import cosmology, units as u, constants as const
from classy import Class

Populating the interactive namespace from numpy and matplotlib


In [2]:
Z_lss = 1091.0
cosmo1 = cosmology.FlatLambdaCDM(71.0, 0.222 + 0.045, Ob0 = 0.045)

In [3]:
def get_class_parameters(cosmology):
    """
    Get CLASS parameters corresponding to an astropy cosmology model.
    """
    class_parameters = {}
    try:
        class_parameters['h'] = cosmology.h
        class_parameters['T_cmb'] = cosmology.Tcmb0.value
        class_parameters['Omega_b'] = cosmology.Ob0
        # CDM = DM - massive neutrinos.
        class_parameters['Omega_cdm'] = cosmology.Om0 - cosmology.Ob0
        class_parameters['Omega_k'] = cosmology.Ok0
        # Dark energy sector. CLASS will calculate whichever of
        # Omega_Lambda/fld/scf we do not provide.
        class_parameters['Omega_scf'] = 0.
        try:
            # Only subclasses of wCDM have the w0 attribute.
            class_parameters['w0_fld'] = cosmology.w0
            class_parameters['wa_fld'] = cosmology.wa
            class_parameters['Omega_Lambda'] = 0.
        except AttributeError:
            class_parameters['Omega_fld'] = 0.
        # Neutrino sector.
        if cosmology.has_massive_nu:
            m_nu = cosmology.m_nu
            try:
                num_massive = len(m_nu)
            except TypeError:
                num_massive = 3
                m_nu = [num_massive * m_nu]
            if num_massive > 3:
                raise RuntimeError(
                    'Cannot calculate cosmology with >3 massive neutrinos.')
            m_ncdm = []
            for mass in m_nu:
                if mass > 0:
                    m_ncdm.append(repr(mass.value))
                else:
                    num_massive -= 1
            if num_massive > 0:
                class_parameters['m_ncdm'] = ','.join(m_ncdm)
        else:
            num_massive = 0
        class_parameters['N_ncdm'] = num_massive
        num_light = 3 - num_massive
        class_parameters['N_ur'] = (3.00641 - num_massive +
            num_light*(cosmology.Neff - 3.00641)/3.)
    except AttributeError:
        raise ValueError('Cosmology is missing required attributes.')
    return class_parameters

In [4]:
cosmo = Class()

In [5]:
cosmo.struct_cleanup()
cosmo.empty()

Here we find the values to maintain for $h^2 \Omega_b$, $h^2 \Omega_m$, and $D_*$.

In [5]:
print('Base numbers:')
cosmo.set(get_class_parameters(cosmo1))
cosmo.compute()
h2b = cosmo.omega_b()
h2m = cosmo.Omega_m() * cosmo.h() ** 2
dist = cosmo.angular_distance(Z_lss)
cosmo.struct_cleanup()
cosmo.empty()
print(h2b)
print(h2m)
print(dist)

Base numbers:
0.0226845
0.1345947
12.8764593039


Values of $h$ between $0.6$ and $0.8$ were tested along with values of $\Omega_b$ and $Omega_m$ rounded to the nearest thousandth.

In [6]:
h = np.linspace(0.6,0.8,num=201)
b = np.around(h2b * h ** -2, decimals = 3)
m = np.around(h2m * h ** -2, decimals = 3)
c = m - b

In [7]:
def params_pos():
    test = {}
    test['N_ncdm'] = 0.0
    test['N_ur'] = 3.04
    test['Omega_Lambda'] = 0.0
    test['Omega_scf'] = 0.0
    test['Omega_k'] = 0.0
    test['w0_fld'] = -1.0 + 0.01
    test['wa_fld'] = 0.5
    test['T_cmb'] = 2.725
    return test
def params_neg():
    test = {}
    test['N_ncdm'] = 0.0
    test['N_ur'] = 3.04
    test['Omega_Lambda'] = 0.0
    test['Omega_scf'] = 0.0
    test['Omega_k'] = 0.0
    test['w0_fld'] = -1.0 - 0.01
    test['wa_fld'] = -0.5
    test['T_cmb'] = 2.725
    return test

Values of $w_0 = -1.0 \pm 0.01$ and $w_a = \pm 0.5$ were chosen to keep the effect of $\epsilon$ small compared to $w_a$.

In [8]:
trial = []
print('w0 = -0.99, wa = 0.5:')
for i in range(201):
    test = params_pos()
    test['h'] = h[i]
    test['Omega_b'] = b[i]
    test['Omega_cdm'] = c[i]
    cosmo.set(test)
    cosmo.compute()
    trial.append(np.abs(dist - cosmo.angular_distance(Z_lss)))
    cosmo.struct_cleanup()
    cosmo.empty()
j = np.argmin(trial)
print("h = {}, Om = {}, Ob = {}".format(h[j],m[j],b[j]))

w0 = -0.99, wa = 0.5:
h = 0.65, Om = 0.319, Ob = 0.054


In [9]:
trial = []
print('w0 = -1.01, wa = -0.5:')
for i in range(201):
    test = params_neg()
    test['h'] = h[i]
    test['Omega_b'] = b[i]
    test['Omega_cdm'] = c[i]
    cosmo.set(test)
    cosmo.compute()
    trial.append(np.abs(dist - cosmo.angular_distance(Z_lss)))
    cosmo.struct_cleanup()
    cosmo.empty()
k = np.argmin(trial)
print("h = {}, Om = {}, Ob = {}".format(h[k],m[k],b[k]))

w0 = -1.01, wa = -0.5:
h = 0.76, Om = 0.233, Ob = 0.039


This test returns optimal parameter values for these two $w(z)$ cosmologies:
 - $w(z) = -0.99 + 0.5 \frac{z}{1+z}$:
  - $h = 0.650$
  - $\Omega_m = 0.319$
  - $\Omega_b = 0.054$
 - $w(z) = -1.01 - 0.5 \frac{z}{1+z}$:
  - $h = 0.760$
  - $\Omega_m = 0.233$
  - $\Omega_b = 0.039$